## 0. 設定

In [1]:
# Do not show warnings
import warnings
warnings.filterwarnings('ignore')

# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib for japanese support
import japanize_matplotlib

# functions for data frame display
from IPython.display import display

# Adjust display options
np.set_printoptions(suppress=True, precision=4)
pd.options.display.float_format = '{:.4f}'.format
pd.set_option("display.max_columns", None)
plt.rcParams["font.size"]=14
random_seed = 123

## 1. データの読み込み

In [2]:
import seaborn as sns

df_titanic = sns.load_dataset("titanic")

columns_t = ['生存', '等室', '性別', '年齢', '兄弟配偶者数',
             '両親子供数', '料金', '乗船港コード', '等室名',
             '男女子供', '成人男子', 'デッキ', '乗船港', '生存可否', '独身']
df_titanic.columns = columns_t

## 2. 不要項目の削除

等室「pclass」と等室名「class」、乗船港コード「embarked」と乗船港「embarked_town」、生存「survived」と生存可否「alive」はそれぞれ同じ意味を持つため、不要な項目を削除する必要がある。

In [3]:
df1 = df_titanic.drop('等室名', axis=1)
df2 = df1.drop('乗船港', axis=1)
df3 = df2.drop('生存可否', axis=1)
display(df3.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港コード,男女子供,成人男子,デッキ,独身
0,0,3,male,22.0000,1,0,7.2500,S,man,True,NaN,False
1,1,1,female,38.0000,1,0,71.2833,C,woman,False,C,False
2,1,3,female,26.0000,0,0,7.9250,S,woman,False,NaN,True
3,1,1,female,35.0000,1,0,53.1000,S,woman,False,C,False
4,0,3,male,35.0000,0,0,8.0500,S,man,True,NaN,True


## 3. 欠損値対応

In [9]:
display(df3.isnull().sum())
display(df3["デッキ"].value_counts())

生存          0
等室          0
性別          0
年齢        177
兄弟配偶者数      0
両親子供数       0
料金          0
乗船港コード      2
男女子供        0
成人男子        0
デッキ       688
独身          0
dtype: int64

C    59
B    47
D    33
E    32
A    15
F    13
G     4
Name: デッキ, dtype: int64

「年齢」、「乗船港コード」、「デッキ」には欠損値があり、データ全体の891件に比べて、「乗船港コード」の２件は少ない比率となるため、行ごとの削除を行う。「年齢」の欠損値が１７７行とかなり多いため、データの平均値で代用。「デッキ」の欠損行数が688行と多く、欠損を意味するダミーコードを振って全行を処理対象にする。

In [8]:
df4 = df3.dropna(subset=['乗船港コード'])
age_average = df4['年齢'].mean()
df5 = df4.fillna({'年齢': age_average})
df6 = df5.replace({'デッキ': {np.nan: 'N'}})
display(df6.isnull().sum())
display(df6.head())

生存        0
等室        0
性別        0
年齢        0
兄弟配偶者数    0
両親子供数     0
料金        0
乗船港コード    0
男女子供      0
成人男子      0
デッキ       0
独身        0
dtype: int64

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港コード,男女子供,成人男子,デッキ,独身
0,0,3,male,22.0000,1,0,7.2500,S,man,True,N,False
1,1,1,female,38.0000,1,0,71.2833,C,woman,False,C,False
2,1,3,female,26.0000,0,0,7.9250,S,woman,False,N,True
3,1,1,female,35.0000,1,0,53.1000,S,woman,False,C,False
4,0,3,male,35.0000,0,0,8.0500,S,man,True,N,True


## 4. 2値ラベルの数値化
２値ラベルの値をとっている項目は、性別「male/female」、成人男子「True/False」、独身「True/False」の３項目

In [10]:
mf_map = {'male':1, 'female':0}

df7 = df6.copy()
df7['性別'] = df7['性別'].map(mf_map)
display(df7.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港コード,男女子供,成人男子,デッキ,独身
0,0,3,1,22.0000,1,0,7.2500,S,man,True,N,False
1,1,1,0,38.0000,1,0,71.2833,C,woman,False,C,False
2,1,3,0,26.0000,0,0,7.9250,S,woman,False,N,True
3,1,1,0,35.0000,1,0,53.1000,S,woman,False,C,False
4,0,3,1,35.0000,0,0,8.0500,S,man,True,N,True


In [13]:
tf_map = {True:1, False:0}
df8 = df7.copy()
df8['成人男子'] = df8['成人男子'].map(tf_map)
display(df8.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港コード,男女子供,成人男子,デッキ,独身
0,0,3,1,22.0000,1,0,7.2500,S,man,1,N,False
1,1,1,0,38.0000,1,0,71.2833,C,woman,0,C,False
2,1,3,0,26.0000,0,0,7.9250,S,woman,0,N,True
3,1,1,0,35.0000,1,0,53.1000,S,woman,0,C,False
4,0,3,1,35.0000,0,0,8.0500,S,man,1,N,True


In [14]:
df9 = df8.copy()
df9['独身'] = df9['独身'].map(tf_map)
display(df9.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港コード,男女子供,成人男子,デッキ,独身
0,0,3,1,22.0000,1,0,7.2500,S,man,1,N,0
1,1,1,0,38.0000,1,0,71.2833,C,woman,0,C,0
2,1,3,0,26.0000,0,0,7.9250,S,woman,0,N,1
3,1,1,0,35.0000,1,0,53.1000,S,woman,0,C,0
4,0,3,1,35.0000,0,0,8.0500,S,man,1,N,1


## 5. 多値ラベルの数値化
多値ラベルの値をとっている項目は、乗船港コード「S/C/Q」、男女子供「man/woman/child」、デッキ「C/B/D/E/A/F/G/N」の３項目である。

In [19]:
w = pd.get_dummies(df9['男女子供'], prefix='男女子供')
display(w.head(10))

,男女子供_child,男女子供_man,男女子供_woman
0,0,1,0
1,0,0,1
2,0,0,1
3,0,0,1
4,0,1,0
5,0,1,0
6,0,1,0
7,1,0,0
8,0,0,1
9,1,0,0


In [20]:
def enc(df, column):
    df_dummy = pd.get_dummies(df[column], prefix=column)
    df_drop = df.drop([column], axis=1)
    df1 = pd.concat([df_drop, df_dummy], axis=1)
    return df1

In [21]:
df10 = enc(df9, '男女子供')
display(df10.head())

df11 = enc(df10, '乗船港コード')
df12 = enc(df11, 'デッキ')
display(df12.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港コード,成人男子,デッキ,独身,男女子供_child,男女子供_man,男女子供_woman
0,0,3,1,22.0000,1,0,7.2500,S,1,N,0,0,1,0
1,1,1,0,38.0000,1,0,71.2833,C,0,C,0,0,0,1
2,1,3,0,26.0000,0,0,7.9250,S,0,N,1,0,0,1
3,1,1,0,35.0000,1,0,53.1000,S,0,C,0,0,0,1
4,0,3,1,35.0000,0,0,8.0500,S,1,N,1,0,1,0


,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,成人男子,独身,男女子供_child,男女子供_man,男女子供_woman,乗船港コード_C,乗船港コード_Q,乗船港コード_S,デッキ_A,デッキ_B,デッキ_C,デッキ_D,デッキ_E,デッキ_F,デッキ_G,デッキ_N
0,0,3,1,22.0000,1,0,7.2500,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1
1,1,1,0,38.0000,1,0,71.2833,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0
2,1,3,0,26.0000,0,0,7.9250,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1
3,1,1,0,35.0000,1,0,53.1000,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0
4,0,3,1,35.0000,0,0,8.0500,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1


## 6. 正規化
機械学習では、絶対値が１前後の数値のほうが精度が高くなることがわかっている。入力データを１前後のものに変換する正規化手法には、normalizationとstandardizationがある。入力変数を最小値が０、最大値が１となるようにするのは、normalizationであり、入力変数が平均０、分散１の正規分布になるように変換するのはstandardizationである。normalizationは外れ値の影響を受けやすく、外れ値を含んでいる可能性がある場合は、standardizationを選択したほうが無難である。なので、画像データのように最大値と最小値が事前にわかっている場合は、normalizationを使うことが多く、その他はstandardizationを使うことが多い。

In [22]:
df13 = df12.copy()
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
df13[['年齢', '料金']]=stdsc.fit_transform(df13[['年齢', '料金']])
display(df13.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,成人男子,独身,男女子供_child,男女子供_man,男女子供_woman,乗船港コード_C,乗船港コード_Q,乗船港コード_S,デッキ_A,デッキ_B,デッキ_C,デッキ_D,デッキ_E,デッキ_F,デッキ_G,デッキ_N
0,0,3,1,-0.5896,1,0,-0.5002,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1
1,1,1,0,0.6448,1,0,0.7889,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0
2,1,3,0,-0.2810,0,0,-0.4866,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1
3,1,1,0,0.4134,1,0,0.4229,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0
4,0,3,1,0.4134,0,0,-0.4841,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1


## ７. その他前処理
「年齢」のような数値データを「１０代」、「２０代」のようなレベル値に分類したり、企業の年間売上のように、企業の規模により様々な値を取りうる数値は対数を取ったほうが良い場合がある。周期的に変化する数値に対しては、三角関数が有効な場合があるなど。こうした高度な前処理は「特徴量エンジニアリング」と呼ばれる。